In [37]:
stage_1_schema = [
'Substrate_stack_sequence',
'ETL_stack_sequence',
'ETL_additives_compounds',
'ETL_deposition_procedure',
'Perovskite_composition_long_form',
'Perovskite_composition_short_form',
'Perovskite_additives_compounds',
'Perovskite_deposition_solvents',
'Perovskite_deposition_procedure',
'Perovskite_deposition_thermal_annealing_temperature',
'Perovskite_deposition_thermal_annealing_time',
'HTL_stack_sequence',
'HTL_additives_compounds',
'HTL_deposition_procedure',
'Backcontact_stack_sequence',
'Backcontact_additives_compounds',
'Backcontact_deposition_procedure',
'Stability_measured',
'Stability_average_over_n_number_of_cells',
'Stability_temperature_range',
'Stability_atmosphere',
'Stability_time_total_exposure',
'Stability_PCE_initial_value',
'Stability_PCE_end_of_experiment',
'Cell_area_measured',
'Cell_number_of_cells_per_substrate',
'Cell_architecture',
'Cell_flexible',
'Cell_semitransparent',
'Cell_semitransparent_wavelength_range',
'Module'
]

In [38]:
a_set = ['Substrate_stack_sequence', 
         'ETL_stack_sequence',
         'ETL_additives_compounds',
         'Perovskite_composition_long_form', 
         'Perovskite_composition_short_form', 
         'Perovskite_additives_compounds',
         'HTL_stack_sequence',
         'HTL_additives_compounds', 
         'Backcontact_stack_sequence',
         'Backcontact_additives_compounds']
b_set = ['Cell_area_measured', 
         'Cell_number_of_cells_per_substrate', 
         'Cell_architecture', 
         'Cell_flexible', 
         'Cell_semitransparent', 
         'Cell_semitransparent_wavelength_range']
c_set = ['ETL_deposition_procedure', 
         'HTL_deposition_procedure',
         'Backcontact_deposition_procedure',
         'Perovskite_deposition_procedure',
         'Perovskite_deposition_solvents', 
         'Perovskite_deposition_thermal_annealing_temperature', 
         'Perovskite_deposition_thermal_annealing_time']
d_set = ['Stability_measured', 
         'Stability_average_over_n_number_of_cells', 
         'Stability_temperature_range', 
         'Stability_atmosphere', 
         'Stability_time_total_exposure', 
         'Stability_PCE_initial_value', 
         'Stability_PCE_end_of_experiment']

In [39]:
def if_float(num_str):
    try:
        float(num_str)
        return True
    except ValueError:
        return False

In [40]:
def single_match(single_answer, single_prediction):
    a_words = single_answer[1].replace('|',';').replace(':',';').replace('>>',';').replace('Spin-coating','spin-coated').split(';')
    p_words = single_prediction[1].replace('|',';').replace(':',';').replace('>>',';').replace('Spin-coating','spin-coated').split(';')
    a_words = [x.strip() for x in a_words]
    p_words = [x.strip() for x in p_words]
    # 在prediction里但不在answer里，则为fp
    # 在answer里但不在prediction里，则为fn
    # 都在则为tp
    tp = []
    fp = []
    fn = []
    if single_answer[0] in ['ETL_stack_sequence', 'HTL_stack_sequence', 'Substrate_stack_sequence']:
        answerSet = set()
        outputSet = set()
        for answer in a_words:
            for ap in answer.split("-"):
                answerSet.add(ap)
        for output in p_words:
            for op in output.split("-"):
                outputSet.add(op)
        a_words = list(answerSet)
        p_words = list(outputSet)
    elif 'procedure' in single_answer[0] or single_answer[0] == 'Stability_atmosphere':
        answerSet = set()
        outputSet = set()
        for answer in a_words:
            answerSet.add(answer.lower())
        for output in p_words:
            outputSet.add(output.lower())
        a_words = list(answerSet)
        p_words = list(outputSet)
    elif 'temperature' in single_answer[0] or 'time' in single_answer[0]:
        answerSet = set()
        outputSet = set()
        for answer in a_words:
            if answer != 'nan' and answer != '' and answer != 'Unknown':
                answerSet.add(str(float(answer)))
            else:
                answerSet.add(answer)
        for output in p_words:
            if output != 'nan' and output!='' and output != 'Unknown':
                outputSet.add(str(float(output)))
            else:
                outputSet.add(output)
        a_words = list(answerSet)
        p_words = list(outputSet)
    tmp_predictions = p_words
    if single_answer[0] in ['Perovskite_composition_long_form', 'Perovskite_composition_short_form']:
        tmp = []
        for p in p_words:
            for aw in a_words:
                if p == aw:
                    tp.append(p)
                else:
                    answer_word_set = set(aw.strip())
                    output_word_set = set(p.strip())
                    if answer_word_set.issubset(output_word_set) or output_word_set.issubset(answer_word_set):
                        tp.append(p)
                        tmp.append(aw)
                        tmp_predictions.remove(p)
                        tmp_predictions.append(aw)
        for p in p_words:
            if p not in tp:
                fp.append(p)
        for aw in a_words:
            if aw not in tmp and aw not in tp:
                fn.append(aw)
    else:
        for p in p_words:
            if p.strip() in a_words:
                tp.append(p.strip())
            else:
                fp.append(p.strip())
        for aw in a_words:
            if aw.strip() not in p_words:
                fn.append(aw.strip())
    return [tp, fp, fn, a_words, tmp_predictions]

In [41]:
def calculate_metrics(tp, fp, fn, ac):
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = precision*recall*2/(precision+recall)
    acc = tp/ac
    return round(precision,4), round(recall,4), round(f1,4), round(acc,4)

In [42]:
def relation(a1, a2, p1, p2):
    aas = []
    pps = []
    for a1_ in a1:
        for a2_ in a2:
            aas.append((a1_, a2_))
    for p1_ in p1:
        for p2_ in p2:
            pps.append((p1_, p2_))
    tp = []
    fp = []
    fn = []
    for pps_ in pps:
        if pps_ in aas:
            tp.append(pps_)
        else:
            fp.append(pps_)
    for aas_ in aas:
        if aas_ not in pps:
            fn.append(aas_)
    return [tp, fp, fn, aas]

In [43]:
def relation_2(a1, a2, a3, a4, p1, p2, p3, p4):
    aas = []
    pps = []
    for a1_ in a1:
        for a2_ in a2:
            for a3_ in a3:
                for a4_ in a4:
                    aas.append((a1_, a2_, a3_, a4_))
    for p1_ in p1:
        for p2_ in p2:
            for p3_ in p3:
                for p4_ in p4:
                    pps.append((p1_, p2_, p3_, p4_))
    tp = []
    fp = []
    fn = []
    for pps_ in pps:
        if pps_ in aas:
            tp.append(pps_)
        else:
            fp.append(pps_)
    for aas_ in aas:
        if aas_ not in pps:
            fn.append(aas_)
    return [tp, fp, fn, aas]

In [44]:
def metrics(answers, predictions):
    keys = ['tp', 'fp', 'fn', 'ans', 'pre']
    store = {'tp':{'total':0},'fp':{'total':0},'fn':{'total':0},'ans':{'total':0}, 'pre':{'total':0}}
    metrics_dict = {}
    for i, answer in enumerate(list(answers.items())):       
        if answer[0] in predictions.keys():
            acc_prediction = (answer[0], predictions[answer[0]])
        else:
            return None
        # print(answer, acc_prediction[1])
        metrics = single_match(answer, acc_prediction)
        for idx in range(4):
            store[keys[idx]]['total'] +=len(metrics[idx])
        if answer[0] in a_set or answer[0] in b_set or answer[0] in c_set or answer[0] in d_set:
        # if answer[0] in b_set:
            for idx in range(5):
                store[keys[idx]][answer[0]]=metrics[idx]
    # information extraction      
    metrics_dict['ie'] = calculate_metrics(store['tp']['total'], store['fp']['total'], store['fn']['total'], store['ans']['total'])
    # relationship extraction: ab, ac, abc-d
    ab_metrics = [0,0,0,0]
    for a in a_set:
        for b in b_set:
            relations = relation(store['ans'][a], store['ans'][b], store['pre'][a], store['pre'][b])
            for i in range(4):
                ab_metrics[i] += len(relations[i])
    metrics_dict['ab'] = calculate_metrics(ab_metrics[0], ab_metrics[1], ab_metrics[2], ab_metrics[3])
    ac_metrics = [0,0,0,0]
    for a in a_set:
        for c in c_set:
            if a.split('_')[0] == c.split('_')[0]:
                relations = relation(store['ans'][a], store['ans'][c], store['pre'][a], store['pre'][c])
                # print(relations)
                for i in range(4):
                    ac_metrics[i] += len(relations[i])
    metrics_dict['ac'] = calculate_metrics(ac_metrics[0], ac_metrics[1], ac_metrics[2], ac_metrics[3])
    abc_d_metrics = [0,0,0,0]
    for a in a_set:
        for b in b_set:
            for c in c_set:
                for d in d_set:
                    if a.split('_')[0] == c.split('_')[0]:
                        relations = relation_2(store['ans'][a], store['ans'][b], store['ans'][c], store['ans'][d], store['pre'][a], store['pre'][b], store['pre'][c], store['pre'][d])
                # print(relations)
                for i in range(4):
                    abc_d_metrics[i] += len(relations[i])
    metrics_dict['abc_d'] = calculate_metrics(abc_d_metrics[0], abc_d_metrics[1], abc_d_metrics[2], abc_d_metrics[3])

    return metrics_dict

# load output 

In [45]:
import json

p = []
a = []
p_text = []
a_text = []
len_schema = stage_1_schema.copy()
len_schema.sort(key = lambda y:len(y),reverse=True)  
# print(len_schema)
with open('../sii40_output.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()
    for i ,d in enumerate(data):
        if d.startswith('prediction'):
            p.append(i)
        if d.startswith('answer'):
            a.append(i)
    for i, ip in enumerate(p):
        tmp = ''
        for d in data[ip:a[i]+1]:
            tmp+=d
        tmp = tmp.replace('Fig. 1. ', '')
        tmp = tmp.replace('<s>', '')
        tmp = tmp.replace('</s>', '')
        tmp_schema = {}
        for ls in len_schema:
            for t in tmp.split('\n'):
                if ls in t:
                    pos = t.find(ls+': ')
                    value = t[pos+len(ls+': '):]
                    if ls not in tmp_schema.keys():   
                        if value.endswith(','):
                            value = value[:-1]
                        if '#' in value:
                            value = value.replace('#', '')
                        tmp_schema[ls] = value.strip()
        p_text.append(tmp_schema)
        # print(tmp_schema)
        # print('\n')
    for i, ia in enumerate(a):
        tmp = ''
        if i!=39:
            end = p[i+1]
        else:
            end = len(data)
        for d in data[ia:end]:
            tmp+=d
        
        tmp = tmp.replace('answer --> ', '')
        tmp = tmp.replace('<s>', '')
        tmp = tmp.replace('</s>', '')
        tmp_schema = {}
        for idx, t in enumerate(tmp.split('\n')[:-1]):
            sche = stage_1_schema[idx]
            if sche in t:
                value = t.replace(sche+': ', '')
                if sche == 'Module':
                    tmp_schema[sche] = value.strip()
                else:
                    tmp_schema[sche] = value[:-1].strip()
        a_text.append(tmp_schema)
        # print(tmp_schema)
        # print('\n')

# IE + RE

In [46]:
# round(precision,4), round(recall,4), round(f1,4), round(acc,4)
def total(metric):
    p = 0
    r = 0
    f = 0
    a = 0
    for i in metric:
        p += i[0]
        r += i[1]
        f += i[2]
        a += i[3]
    precision = p/len(metric)
    recall = r/len(metric)
    f1 = f/len(metric)
    acc = a/len(metric)
    return round(precision,4), round(recall,4), round(f1,4), round(acc,4)

ie = []
ab = []
ac = []
abc_d = []
for i, p in enumerate(p_text):
    # print(p)
    # print(a_text[i])
    m = metrics(a_text[i], p)
    if m:
        ie.append(m['ie'])
        ab.append(m['ab'])
        ac.append(m['ac'])
        abc_d.append(m['abc_d'])
print('ie_total', total(ie))
print('ab_relation', total(ab))
print('ac_relation', total(ac))
print('abc_d_relation', total(abc_d))


ie_total (0.8834, 0.8611, 0.8714, 0.8611)
ab_relation (0.7886, 0.7531, 0.7681, 0.7531)
ac_relation (0.7263, 0.6722, 0.6942, 0.6722)
abc_d_relation (0.7197, 0.6584, 0.6823, 0.6584)


# each set IE score

In [47]:
def metrics_single(answers, predictions, set_name):
    keys = ['tp', 'fp', 'fn', 'ans', 'pre']
    store = {'tp':{'total':0},'fp':{'total':0},'fn':{'total':0},'ans':{'total':0}, 'pre':{'total':0}}
    metrics_dict = {}
    for i, answer in enumerate(list(answers.items())): 
        if answer[0] in set_name:
            if answer[0] in predictions.keys():
                acc_prediction = (answer[0], predictions[answer[0]])
            else:
                return None
            metrics = single_match(answer, acc_prediction)
            for idx in range(4):
                store[keys[idx]]['total'] +=len(metrics[idx])
            for idx in range(5):
                store[keys[idx]][answer[0]]=metrics[idx]     
    metrics_dict['ie'] = calculate_metrics(store['tp']['total'], store['fp']['total'], store['fn']['total'], store['ans']['total'])
    return metrics_dict

In [48]:
# round(precision,4), round(recall,4), round(f1,4), round(acc,4)
a = []
b = []
c = []
d = []
for i, p in enumerate(p_text):
    m = metrics_single(a_text[i], p, a_set)
    if m:
        a.append(m['ie'])
for i, p in enumerate(p_text):
    m = metrics_single(a_text[i], p, b_set)
    if m:
        b.append(m['ie'])
for i, p in enumerate(p_text):
    m = metrics_single(a_text[i], p, c_set)
    if m:
        c.append(m['ie'])
for i, p in enumerate(p_text):
    m = metrics_single(a_text[i], p, d_set)
    if m:
        d.append(m['ie'])

print('a_ie', total(a))
print('b_ie', total(b))
print('c_ie', total(c))
print('d_ie', total(d))

a_ie (0.8354, 0.8123, 0.821, 0.8123)
b_ie (0.902, 0.902, 0.902, 0.902)
c_ie (0.828, 0.7831, 0.8014, 0.7831)
d_ie (0.9796, 0.9796, 0.9796, 0.9796)


# deduction, normalization, summarization

In [49]:
deduction = ['Cell_area_measured', 'Cell_architecture', 'Cell_semitransparent']
normalization = ['Perovskite_deposition_thermal_annealing_time', 'Stability_time_total_exposure', 'Cell_area_measured']
summarization=['Substrate_stack_sequence', 'ETL_stack_sequence', 'Perovskite_composition_short_form', 'HTL_stack_sequence']

In [64]:
with open('original_text.json', 'r', encoding='utf-8') as f:
    prompts = json.load(f)

In [65]:
import json

p = []
a = []
p_text = []
a_text = []
len_schema = stage_1_schema.copy()
len_schema.sort(key = lambda y:len(y),reverse=True)  
# print(len_schema)
with open('../sii40_output.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()
    for i ,d in enumerate(data):
        if d.startswith('prediction'):
            p.append(i)
        if d.startswith('answer'):
            a.append(i)
    for i, ip in enumerate(p):
        tmp = ''
        for d in data[ip:a[i]+1]:
            tmp+=d
        tmp = tmp.replace('Fig. 1. ', '')
        tmp = tmp.replace('<s>', '')
        tmp = tmp.replace('</s>', '')
        tmp_schema = {}
        for ls in len_schema:
            for t in tmp.split('\n'):
                if ls in t:
                    pos = t.find(ls+': ')
                    value = t[pos+len(ls+': '):]
                    if ls not in tmp_schema.keys():   
                        if value.endswith(','):
                            value = value[:-1]
                        if '#' in value:
                            value = value.replace('#', '')
                        tmp_schema[ls] = value.strip()
        p_text.append(tmp_schema)
        # print(tmp_schema)
        # print('\n')
    for i, ia in enumerate(a):
        tmp = ''
        if i!=39:
            end = p[i+1]
        else:
            end = len(data)
        for d in data[ia:end]:
            tmp+=d
        
        tmp = tmp.replace('answer --> ', '')
        tmp = tmp.replace('<s>', '')
        tmp = tmp.replace('</s>', '')
        tmp_schema = {}
        for idx, t in enumerate(tmp.split('\n')[:-1]):
            sche = stage_1_schema[idx]
            if sche in t:
                value = t.replace(sche+': ', '')
                if sche == 'Module':
                    tmp_schema[sche] = value.strip()
                else:
                    tmp_schema[sche] = value[:-1].strip()
        a_text.append(tmp_schema)
        # print(tmp_schema)
        # print('\n')
print(len(p_text))
print(p_text[0])

40
{'Perovskite_deposition_thermal_annealing_temperature': '90', 'Perovskite_deposition_thermal_annealing_time': '40', 'Stability_average_over_n_number_of_cells': '1', 'Cell_semitransparent_wavelength_range': 'nan; nan', 'Cell_number_of_cells_per_substrate': '0', 'Perovskite_composition_short_form': 'MAPbI', 'Perovskite_composition_long_form': 'MAPbI3', 'Backcontact_deposition_procedure': 'Evaporation', 'Perovskite_deposition_procedure': 'Spin-coating', 'Backcontact_additives_compounds': '', 'Stability_PCE_end_of_experiment': '', 'Perovskite_additives_compounds': '', 'Perovskite_deposition_solvents': 'Unknown', 'Stability_time_total_exposure': '', 'Stability_temperature_range': 'nan; nan', 'Stability_PCE_initial_value': '', 'Backcontact_stack_sequence': 'Ag', 'Substrate_stack_sequence': 'SLG | ITO', 'ETL_deposition_procedure': 'Spin-coating', 'HTL_deposition_procedure': 'Spin-coating', 'ETL_additives_compounds': 'BCP', 'HTL_additives_compounds': '', 'Stability_atmosphere': 'Unknown', '

In [66]:
d_f_t = 0
d_f = 0
d_n_t = 0
d_n = 0
d_s_t = 0
d_s = 0
for i, p in enumerate(p_text):
    for idx, a in enumerate(a_text[i].keys()):
        if a in deduction:
            if a in p.keys() and a_text[i][a] not in prompts[i]:
                d_f_t += 1
                if p[a]==a_text[i][a]:
                    d_f += 1
        if a in normalization:
            if a in p.keys() and a_text[i][a] not in prompts[i]:
                d_n_t += 1
                if p[a]==a_text[i][a]:
                    d_n += 1
                else:
                    # print(p[a], a_text[i][a])
                    a_words = a_text[i][a].replace('; ',' >> ').replace(' | ', ' >> ').split(' >> ')
                    p_words = p[a].replace('; ',' >> ').replace("''", '').split(' >> ')
                    for pw in p_words:
                        if pw in a_words:
                            d_n += 1
                        else:
                            # print(a_words,p_words)
                            for aw in a_words:
                                if if_float(pw) and if_float(aw):
                                    if round(float(pw),1) == round(float(aw),1):
                                        # print('sub', pw, aw)
                                        d_n += 1
        if a in summarization:
            if a in p.keys(): 
                a_words = a_text[i][a].replace('|',';').replace(':',';').replace('>>',';').replace('Spin-coating','spin-coated').split(';')
                a_words = [x.strip() for x in a_words]
                p_words = p[a].replace('|',';').replace(':',';').replace('>>',';').replace('Spin-coating','spin-coated').split(';')
                p_words = [x.strip() for x in p_words] 
                for aw in a_words:
                    if aw not in prompts[i]:
                        d_s_t += 1
                        if aw in p_words:
                            d_s += 1

In [67]:
print(d_f, d_f_t, d_f/d_f_t)
print(d_n, d_n_t, d_n/d_n_t)
print(d_s, d_s_t, d_s/d_s_t)



53 70 0.7571428571428571
29 41 0.7073170731707317
116 154 0.7532467532467533


In [ ]:
'''
# GPT-3 finetune result
fine_data = []
with open('test_output40.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()
    for d in data:
        if d.startswith('output'):
            fine_data.append(d[:-1].replace('output:  ', ''))
p_text = []
for fd in fine_data:
    pos = []
    tmp_dict = {}
    for i in stage_1_schema:
        pos.append(fd.find(i))
    for i, e in enumerate(stage_1_schema):
        if i!=len(pos)-1:
            end = pos[i+1]-2
        else:
            end = -1
        if end == -1:
            content = fd[pos[i]+2+len(e):]
        else:
            content = fd[pos[i]+2+len(e):end]
        tmp_dict[e] = content.replace("''", '')
    p_text.append(tmp_dict)
# print(len(p_text))
# print(p_text[0])
'''